In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import country_converter as coco
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
label_encoder = LabelEncoder()
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, classification_report, recall_score, precision_recall_curve
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
#pues nada, vamos a empezar. Primero abrire el csv y hechare un vistazo
train = pd.read_csv("../Dataframes/train.csv", sep=",")
train

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,89,973,Acer,Chromebook C738T-C2EJ,2 in 1 Convertible,11.6,IPS Panel Touchscreen 1366x768,Intel Celeron Dual Core N3060 1.6GHz,4GB,32GB Flash Storage,Intel HD Graphics 400,Chrome OS,1.25kg,389.00
908,716,1018,Toshiba,Portege Z30-C-16H,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.2kg,1195.00
909,517,1071,Acer,Aspire ES1-523,Notebook,15.6,1366x768,AMD A8-Series 7410 2.2GHz,8GB,1TB HDD,AMD Radeon R5,Windows 10,2.4kg,449.00
910,891,965,Dell,Latitude 7480,Ultrabook,14.0,Full HD 1920x1080,Intel Core i7 7600U 2.8GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.36kg,1775.00


In [3]:
#Bien, no tenemos nulls
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                912 non-null    int64  
 1   laptop_ID         912 non-null    int64  
 2   Company           912 non-null    object 
 3   Product           912 non-null    object 
 4   TypeName          912 non-null    object 
 5   Inches            912 non-null    float64
 6   ScreenResolution  912 non-null    object 
 7   Cpu               912 non-null    object 
 8   Ram               912 non-null    object 
 9   Memory            912 non-null    object 
 10  Gpu               912 non-null    object 
 11  OpSys             912 non-null    object 
 12  Weight            912 non-null    object 
 13  Price_euros       912 non-null    float64
dtypes: float64(2), int64(2), object(10)
memory usage: 99.9+ KB


In [4]:
train.describe()

,id,laptop_ID,Inches,Price_euros
count,912.000000,912.000000,912.000000,912.000000
mean,645.091009,666.192982,15.011404,1108.122873
std,380.313813,384.873846,1.411744,714.597741
min,0.000000,1.000000,10.100000,174.000000
25%,319.750000,330.500000,14.000000,589.000000
50%,636.500000,673.500000,15.600000,949.000000
75%,980.250000,998.500000,15.600000,1458.500000
max,1302.000000,1320.000000,18.400000,6099.000000


In [5]:
#sns.pairplot(train, hue="Price_euros");
#sns.heatmap(train.corr(), annot=True, cmap="coolwarm")

#no me aportan ningun dato interesante, voy a manipular la DF.
#Posiblemente se deba a los componentes, ahora mismo no voy a poder ver nada.

In [6]:
#Empezemos separando las columnas de texto segun su contenido.

In [7]:
Company = []
for topic in train['Company']:
    for palabra in topic.split(" "):
        Company.append(palabra)

Typename = []
for topic in train['TypeName']:
    for palabra in topic.split(" "):
        Typename.append(palabra)

Cpu = []
for topic in train['Cpu']:
    for palabra in topic.split(" "):
        Cpu.append(palabra)

Ram = []
for topic in train['Ram']:
    for palabra in topic.split(" "):
        Ram.append(palabra)

Memory = []
for topic in train['Memory']:
    for palabra in topic.split(" "):
        Memory.append(palabra)

Gpu = []
for topic in train['Gpu']:
    for palabra in topic.split(" "):
        Gpu.append(palabra)

OpSys = []
for topic in train['OpSys']:
    for palabra in topic.split(" "):
        OpSys.append(palabra)
        
#Weight siguen siendo datos, por si acaso vamos a mirarla, aunque no la incluire.

Weight = []
for topic in train['Weight']:
    for palabra in topic.split(" "):
        Weight.append(palabra)

In [8]:
pd.Series(Company).value_counts().sort_values(ascending=False)

Lenovo       208
Dell         207
HP           189
Asus         108
Acer          75
Toshiba       37
MSI           34
Apple         17
Mediacom       7
Samsung        6
Razer          6
Fujitsu        3
Microsoft      3
Vero           3
Xiaomi         3
Google         2
Chuwi          2
LG             1
Huawei         1
Name: count, dtype: int64

In [9]:
pd.Series(Memory).value_counts().sort_values(ascending=False)

SSD        589
HDD        400
256GB      361
1TB        289
+          140
           140
128GB      120
500GB      102
512GB       92
Flash       54
Storage     54
32GB        33
2TB         18
64GB        13
1.0TB       10
Hybrid       9
16GB         7
180GB        5
240GB        1
8GB          1
Name: count, dtype: int64

In [10]:
pd.Series(Cpu).value_counts().sort_values(ascending=False)

Intel       872
Core        856
i7          351
i5          305
2.5GHz      206
           ... 
v5            1
Z8350         1
X5-Z8350      1
x5-Z8300      1
9000          1
Name: count, Length: 130, dtype: int64

In [11]:
pd.Series(Ram).value_counts().sort_values(ascending=False)

8GB     434
4GB     274
16GB    125
6GB      32
2GB      16
12GB     15
32GB     14
24GB      2
Name: count, dtype: int64

In [12]:
pd.Series(Gpu).value_counts().sort_values(ascending=False).head(15)

Graphics    530
Intel       522
HD          465
Nvidia      268
GeForce     247
620         246
GTX         158
520         150
AMD         122
Radeon      117
1050         64
UHD          45
R5           40
940MX        32
1060         31
Name: count, dtype: int64

In [13]:
df = train.copy()

In [14]:
train = pd.read_csv("../DataFrames/train.csv", sep=",")
train
df_dummies = pd.get_dummies(train['TypeName'])
df = pd.concat([df, df_dummies], axis = 1)
df_dummies = pd.get_dummies(train['Cpu'])
df = pd.concat([df, df_dummies], axis = 1)
df_dummies = pd.get_dummies(train['Company'])
df["Lenovo"] = df_dummies["Lenovo"]
df["Dell"] = df_dummies["Dell"]
df["HP"] = df_dummies["HP"]
df["Asus"] = df_dummies["Asus"]
df["Acer"] = df_dummies["Acer"]

df["Intel Core i7"] = np.where(df["Cpu"].str.contains("Intel Core i7"), True, False)
df["Intel Core i5"] = np.where(df["Cpu"].str.contains("Intel Core i5"), True, False)
df["Intel Core i3"] = np.where(df["Cpu"].str.contains("Intel Core i3"), True, False)
df["CPU_GTX"] = np.where(df["Cpu"].str.contains("GTX"), True, False)

"""df["256GB"] = np.where(df["Memory"].str.contains("256GB"), True, False)
df["512GB"] = np.where(df["Memory"].str.contains("512GB"), True, False)
df["1TB"] = np.where(df["Memory"].str.contains("1TB"), True, False)
df["HDD"] = np.where(df["Memory"].str.contains("HDD"), True, False)
df["SSD"] = np.where(df["Memory"].str.contains("SSD"), True, False)
df["Flash"] = np.where(df["Memory"].str.contains("Flash"), True, False)
df["Hybrid"] = np.where(df["Memory"].str.contains("Hybrid"), True,False)"""

df["Gpu_Amd"] = np.where(df["Gpu"].str.contains("AMD"), True, False)
df["Nvidia_Amd"] = np.where(df["Gpu"].str.contains("Nvidia"), True, False)
df["Nvidia_Geforce"] = np.where(df["Gpu"].str.contains("Geforce"), True, False)
df["Nvidia_graphics"] = np.where(df["Gpu"].str.contains("Graphics"), True, False)
df["Nvidia_HD"] = np.where(df["Gpu"].str.contains("HD"), True, False)
df["Intel_GPU"] = np.where(df["Gpu"].str.contains("Intel"), True, False)

df["ppi"] = np.where(df["ScreenResolution"].str.contains("PPi"), True, False)



In [15]:
df['Memory'] = df['Memory'].astype(str).replace('.0', '', regex=True)
df["Memory"] = df["Memory"].str.replace('GB', '')
df["Memory"] = df["Memory"].str.replace('TB', '000')
new = df["Memory"].str.split("+", n = 1, expand = True)
df["first"]= new[0]
df["first"]=df["first"].str.strip()
df["second"]= new[1]
df["Layer1HDD"] = df["first"].apply(lambda x: 1 if "HDD" in x else 0)
df["Layer1SSD"] = df["first"].apply(lambda x: 1 if "SSD" in x else 0)
df["Layer1Hybrid"] = df["first"].apply(lambda x: 1 if "Hybrid" in x else 0)
df["Layer1Flash_Storage"] = df["first"].apply(lambda x: 1 if "Flash Storage" in x else 0)
df['first'] = df['first'].str.replace(r'D', '')
df['first'] = df['first'].str.replace(r'Hybrid', '')
df['first'] = df['first'].str.replace(r'Flash Storage', '')
df['first'] = df['first'].str.replace(r'S', '')
df['first'] = df['first'].str.replace(r'H', '')

df["first"].replace("", pd.NA, inplace=True)
df["first"].fillna(0, inplace=True)
df["second"].fillna("0", inplace = True)
df["Layer2HDD"] = df["second"].apply(lambda x: 1 if "HDD" in x else 0)
df["Layer2SSD"] = df["second"].apply(lambda x: 1 if "SSD" in x else 0)
df["Layer2Hybrid"] = df["second"].apply(lambda x: 1 if "Hybrid" in x else 0)
df["Layer2Flash_Storage"] = df["second"].apply(lambda x: 1 if "Flash Storage" in x else 0)
df['second'] = df['second'].str.replace(r'D', '')
#binary encoding

df["Layer2HDD"] = df["second"].apply(lambda x: 1 if "HDD" in x else 0)
df["Layer2SSD"] = df["second"].apply(lambda x: 1 if "SSD" in x else 0)
df["Layer2Hybrid"] = df["second"].apply(lambda x: 1 if "Hybrid" in x else 0)
df["Layer2Flash_Storage"] = df["second"].apply(lambda x: 1 if "Flash Storage" in x else 0)
df["Layer2Flash_Storage"] = df["second"].apply(lambda x: 1 if "Flash Storage" in x else 0)
#only keep integert(digits)
df['second'] = df['second'].str.replace(r'D', '')
df['second'] = df['second'].str.replace(r'Hybrid', '')
df['second'] = df['second'].str.replace(r'Flash Storage', '')
df['second'] = df['second'].str.replace(r'H', '')
df['second'] = df['second'].str.replace(r'S', '')

df['second'] = df['second'].str.replace(r' ', '')
df["second"].replace("", pd.NA, inplace=True)
df["second"].fillna(0, inplace=True)

df["first"] = df["first"].astype(int)
df["second"] = df["second"].astype(int)
#finalize the columns by keeping value
df["HDD"]=(df["first"]*df["Layer1HDD"]+df["second"]*df["Layer2HDD"])
df["SSD"]=(df["first"]*df["Layer1SSD"]+df["second"]*df["Layer2SSD"])
df["Hybrid"]=(df["first"]*df["Layer1Hybrid"]+df["second"]*df["Layer2Hybrid"])
df["Flash_Storage"]=(df["first"]*df["Layer1Flash_Storage"]+df["second"]*df["Layer2Flash_Storage"])
#Drop the un required columns
df.drop(columns=['first', 'second', 'Layer1HDD', 'Layer1SSD', 'Layer1Hybrid',
       'Layer1Flash_Storage', 'Layer2HDD', 'Layer2SSD', 'Layer2Hybrid',
       'Layer2Flash_Storage'],inplace=True)

In [16]:
def findXresolution(s):
  return s.split()[-1].split("x")[0]
def findYresolution(s):
  return s.split()[-1].split("x")[1]
#finding the x_res and y_res from screen resolution
df['X_res'] = df['ScreenResolution'].apply(lambda x: findXresolution(x))
df['Y_res'] = df['ScreenResolution'].apply(lambda y: findYresolution(y))
df['X_res'] = df['X_res'].astype('int')
df['Y_res'] = df['Y_res'].astype('int')
df['ppi'] = (((df['X_res']**2) + (df['Y_res']**2))**0.5/df['Inches']).astype('float')


In [17]:
def cat_os(inp):
    if inp == 'Windows 10' or inp == 'Windows 7' or inp == 'Windows 10 S':
        return 'Windows'
    elif inp == 'macOS' or inp == 'Mac OS X':
        return 'Mac'
    else:
        return 'Others/No OS/Linux'
df['SO'] = df['OpSys'].apply(cat_os)
df['SO'] = label_encoder.fit_transform(df["SO"])

In [18]:
df["Ram"] = df["Ram"].str.replace("GB","").astype("int64")
df["HDD + SSD"] = np.where(df["Memory"].str.contains("SSD") & df["Memory"].str.contains("HDD"), True, False)

In [19]:
df_final = df.drop(["id", "laptop_ID", "Inches", "Company", "Product", "TypeName", "ScreenResolution","Cpu", "Memory", "Gpu","OpSys", "Weight"], axis=1)

In [20]:
test = pd.read_csv("test.csv", sep=",")
df = test.copy()

In [21]:
df_dummies = pd.get_dummies(test['TypeName'])
df = pd.concat([df, df_dummies], axis = 1)
df_dummies = pd.get_dummies(test['Company'])
df = pd.concat([df, df_dummies], axis = 1)
df["Lenovo"] = df_dummies["Lenovo"]
df["Dell"] = df_dummies["Dell"]
df["HP"] = df_dummies["HP"]
df["Asus"] = df_dummies["Asus"]
df["Acer"] = df_dummies["Acer"]

df["Intel Core i7"] = np.where(df["Cpu"].str.contains("Intel Core i7"), True, False)
df["Intel Core i5"] = np.where(df["Cpu"].str.contains("Intel Core i5"), True, False)
df["Intel Core i3"] = np.where(df["Cpu"].str.contains("Intel Core i3"), True, False)
df["CPU_GTX"] = np.where(df["Cpu"].str.contains("GTX"), True, False)

"""df["256GB"] = np.where(df["Memory"].str.contains("256GB"), True, False)
df["512GB"] = np.where(df["Memory"].str.contains("512GB"), True, False)
df["1TB"] = np.where(df["Memory"].str.contains("1TB"), True, False)
df["HDD"] = np.where(df["Memory"].str.contains("HDD"), True, False)
df["SSD"] = np.where(df["Memory"].str.contains("SSD"), True, False)
df["Flash"] = np.where(df["Memory"].str.contains("Flash"), True, False)
df["Hybrid"] = np.where(df["Memory"].str.contains("Hybrid"), True,False)"""

df["Gpu_Amd"] = np.where(df["Gpu"].str.contains("AMD"), True, False)
df["Nvidia_Amd"] = np.where(df["Gpu"].str.contains("Nvidia"), True, False)
df["Nvidia_Geforce"] = np.where(df["Gpu"].str.contains("Geforce"), True, False)
df["Nvidia_graphics"] = np.where(df["Gpu"].str.contains("Graphics"), True, False)
df["Nvidia_HD"] = np.where(df["Gpu"].str.contains("HD"), True, False)
df["Intel_CPU"] = np.where(df["Gpu"].str.contains("Intel"), True, False)

df["HDD + SSD"] = np.where(df["Memory"].str.contains("SSD") & df["Memory"].str.contains("HDD"), True, False)

df["ppi"] = np.where(df["ScreenResolution"].str.contains("PPi"), True, False)




In [22]:
df['Memory'] = df['Memory'].astype(str).replace('.0', '', regex=True)
df["Memory"] = df["Memory"].str.replace('GB', '')
df["Memory"] = df["Memory"].str.replace('TB', '000')
new = df["Memory"].str.split("+", n = 1, expand = True)
df["first"]= new[0]
df["first"]=df["first"].str.strip()
df["second"]= new[1]
df["Layer1HDD"] = df["first"].apply(lambda x: 1 if "HDD" in x else 0)
df["Layer1SSD"] = df["first"].apply(lambda x: 1 if "SSD" in x else 0)
df["Layer1Hybrid"] = df["first"].apply(lambda x: 1 if "Hybrid" in x else 0)
df["Layer1Flash_Storage"] = df["first"].apply(lambda x: 1 if "Flash Storage" in x else 0)
df['first'] = df['first'].str.replace(r'D', '')
df['first'] = df['first'].str.replace(r'Hybrid', '')
df['first'] = df['first'].str.replace(r'Flash Storage', '')
df['first'] = df['first'].str.replace(r'S', '')
df['first'] = df['first'].str.replace(r'H', '')

df["first"].replace("", pd.NA, inplace=True)
df["first"].fillna(0, inplace=True)
df["second"].fillna("0", inplace = True)
df["Layer2HDD"] = df["second"].apply(lambda x: 1 if "HDD" in x else 0)
df["Layer2SSD"] = df["second"].apply(lambda x: 1 if "SSD" in x else 0)
df["Layer2Hybrid"] = df["second"].apply(lambda x: 1 if "Hybrid" in x else 0)
df["Layer2Flash_Storage"] = df["second"].apply(lambda x: 1 if "Flash Storage" in x else 0)
df['second'] = df['second'].str.replace(r'D', '')
#binary encoding

df["Layer2HDD"] = df["second"].apply(lambda x: 1 if "HDD" in x else 0)
df["Layer2SSD"] = df["second"].apply(lambda x: 1 if "SSD" in x else 0)
df["Layer2Hybrid"] = df["second"].apply(lambda x: 1 if "Hybrid" in x else 0)
df["Layer2Flash_Storage"] = df["second"].apply(lambda x: 1 if "Flash Storage" in x else 0)
df["Layer2Flash_Storage"] = df["second"].apply(lambda x: 1 if "Flash Storage" in x else 0)
#only keep integert(digits)
df['second'] = df['second'].str.replace(r'D', '')
df['second'] = df['second'].str.replace(r'Hybrid', '')
df['second'] = df['second'].str.replace(r'Flash Storage', '')
df['second'] = df['second'].str.replace(r'H', '')
df['second'] = df['second'].str.replace(r'S', '')

df['second'] = df['second'].str.replace(r' ', '')
df["second"].replace("", pd.NA, inplace=True)
df["second"].fillna(0, inplace=True)

df["first"] = df["first"].astype(int)
df["second"] = df["second"].astype(int)
#finalize the columns by keeping value
df["HDD"]=(df["first"]*df["Layer1HDD"]+df["second"]*df["Layer2HDD"])
df["SSD"]=(df["first"]*df["Layer1SSD"]+df["second"]*df["Layer2SSD"])
df["Hybrid"]=(df["first"]*df["Layer1Hybrid"]+df["second"]*df["Layer2Hybrid"])
df["Flash_Storage"]=(df["first"]*df["Layer1Flash_Storage"]+df["second"]*df["Layer2Flash_Storage"])
#Drop the un required columns
df.drop(columns=['first', 'second', 'Layer1HDD', 'Layer1SSD', 'Layer1Hybrid',
       'Layer1Flash_Storage', 'Layer2HDD', 'Layer2SSD', 'Layer2Hybrid',
       'Layer2Flash_Storage'],inplace=True)

In [23]:
df['X_res'] = df['ScreenResolution'].apply(lambda x: findXresolution(x))
df['Y_res'] = df['ScreenResolution'].apply(lambda y: findYresolution(y))
df['X_res'] = df['X_res'].astype('int')
df['Y_res'] = df['Y_res'].astype('int')
df['ppi'] = (((df['X_res']**2) + (df['Y_res']**2))**0.5/df['Inches']).astype('float')


In [24]:
df['SO'] = df['OpSys'].apply(cat_os)

df['SO'] = label_encoder.fit_transform(df["SO"])

In [25]:
df["Ram"] = df["Ram"].str.replace("GB","").astype("int64")
test_final = df.drop(["id", "laptop_ID", "Company", "Product", "TypeName", "ScreenResolution","Cpu", "Memory", "Gpu","OpSys", "Weight"], axis=1)

In [26]:
test_final = df.drop(["id", "laptop_ID", "Company", "Product","TypeName", "ScreenResolution","Cpu", "Memory", "Gpu","OpSys", "Weight"], axis=1)

In [27]:
train

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,89,973,Acer,Chromebook C738T-C2EJ,2 in 1 Convertible,11.6,IPS Panel Touchscreen 1366x768,Intel Celeron Dual Core N3060 1.6GHz,4GB,32GB Flash Storage,Intel HD Graphics 400,Chrome OS,1.25kg,389.00
908,716,1018,Toshiba,Portege Z30-C-16H,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.2kg,1195.00
909,517,1071,Acer,Aspire ES1-523,Notebook,15.6,1366x768,AMD A8-Series 7410 2.2GHz,8GB,1TB HDD,AMD Radeon R5,Windows 10,2.4kg,449.00
910,891,965,Dell,Latitude 7480,Ultrabook,14.0,Full HD 1920x1080,Intel Core i7 7600U 2.8GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.36kg,1775.00


In [28]:
test_final

,Inches,Ram,2 in 1 Convertible,Gaming,Netbook,Notebook,Ultrabook,Workstation,Acer,Apple,...,Intel_CPU,HDD + SSD,ppi,HDD,SSD,Hybrid,Flash_Storage,X_res,Y_res,SO
0,13.3,16,False,False,False,False,True,False,False,False,...,True,False,331.264236,0,512,0,0,3840,2160,2
1,15.6,8,False,False,False,True,False,False,True,False,...,False,False,100.454670,0,256,0,0,1366,768,2
2,15.6,4,False,False,False,True,False,False,True,False,...,True,False,100.454670,0,0,0,0,1366,768,1
3,14.0,8,False,False,False,True,False,False,False,False,...,True,False,157.350512,0,256,0,0,1920,1080,2
4,13.3,8,False,False,False,True,False,False,False,False,...,True,False,165.632118,0,256,0,0,1920,1080,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,15.6,8,False,True,False,False,False,False,False,False,...,False,False,141.211998,0,256,0,0,1920,1080,1
387,15.6,16,False,True,False,False,False,False,False,False,...,False,False,141.211998,0,512,0,0,1920,1080,2
388,15.6,4,False,False,False,True,False,False,False,False,...,True,False,100.454670,0,0,0,0,1366,768,1
389,15.6,4,False,False,False,True,False,False,False,False,...,False,False,141.211998,0,0,0,0,1920,1080,2


In [29]:
columnas = df_final.columns.intersection(test_final.columns)
df_final = df_final[columnas]
columnas = test_final.columns.intersection(df_final.columns)
test_final = test_final[columnas]
df_final = df_final.loc[:,~df_final.columns.duplicated()]
test_final = test_final.loc[:,~test_final.columns.duplicated()]


In [30]:
df_final = df_final.reindex(sorted(df_final.columns), axis=1)
test_final =test_final.reindex(sorted(test_final.columns), axis=1)

In [31]:
test_final

,2 in 1 Convertible,Acer,Asus,CPU_GTX,Dell,Flash_Storage,Gaming,Gpu_Amd,HDD,HDD + SSD,...,Nvidia_HD,Nvidia_graphics,Ram,SO,SSD,Ultrabook,Workstation,X_res,Y_res,ppi
0,False,False,False,False,False,0,False,False,0,False,...,True,True,16,2,512,True,False,3840,2160,331.264236
1,False,True,False,False,False,0,False,True,0,False,...,False,False,8,2,256,False,False,1366,768,100.454670
2,False,True,False,False,False,0,False,False,0,False,...,True,True,4,1,0,False,False,1366,768,100.454670
3,False,False,False,False,False,0,False,False,0,False,...,True,True,8,2,256,False,False,1920,1080,157.350512
4,False,False,False,False,False,0,False,False,0,False,...,True,True,8,2,256,False,False,1920,1080,165.632118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,False,False,False,False,False,0,True,False,0,False,...,False,False,8,1,256,False,False,1920,1080,141.211998
387,False,False,False,False,False,0,True,False,0,False,...,False,False,16,2,512,False,False,1920,1080,141.211998
388,False,False,False,False,False,0,False,False,0,False,...,True,True,4,1,0,False,False,1366,768,100.454670
389,False,False,False,False,False,0,False,True,0,False,...,False,False,4,2,0,False,False,1920,1080,141.211998


In [32]:
X = df_final
y = train["Price_euros"]

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 20)
lr = LinearRegression()
lr.fit(X_train, y_train)
print("Total features shape:", X.shape)
print("Train features shape:", X_train.shape)
print("Train target shape:", y_train.shape)
print("Test features shape:", X_test.shape)
print("Test target shape:", y_test.shape)

Total features shape: (912, 30)
Train features shape: (729, 30)
Train target shape: (729,)
Test features shape: (183, 30)
Test target shape: (183,)


In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 20)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scal = scaler.transform(X_train)
X_test_scal = scaler.transform(X_test)
poly = PolynomialFeatures(degree=2)
poly.fit(X_train_scal)
X_poly_train = poly.transform(X_train_scal)
X_poly_test = poly.transform(X_test_scal)

pol_reg4_reg = Ridge(alpha=0.4)
pol_reg4_reg.fit(X_poly_train, y_train)

predictions = pol_reg4_reg.predict(X_poly_test)

print("MAE test", mean_absolute_error(y_test, predictions))
print("Total features shape:", X.shape)
print("Train features shape:", X_train.shape)
print("Train target shape:", y_train.shape)
print("Test features shape:", X_test.shape)
print("Test target shape:", y_test.shape)

MAE test 223.43667694799524
Total features shape: (912, 30)
Train features shape: (729, 30)
Train target shape: (729,)
Test features shape: (183, 30)
Test target shape: (183,)


In [35]:
predictions = lr.predict(X_test)
predictions

array([ 690.1640625, 2466.0078125, 1081.5703125,  839.515625 ,
        807.984375 , 1339.515625 ,  805.71875  , 1782.265625 ,
        667.921875 ,  540.15625  ,  981.25     ,  762.03125  ,
       1274.453125 ,  347.7265625,  327.7421875, 1324.859375 ,
        832.3125   ,  412.       , 2245.171875 , 1767.4140625,
       1394.796875 ,  922.125    ,  443.03125  ,  981.5234375,
        336.6875   ,  577.0703125,  664.328125 , 1719.75     ,
        770.140625 , 2021.2109375, 1158.9765625,  292.5703125,
       1013.5      ,  750.6875   , 2516.7890625, 1432.7265625,
       1806.15625  ,  928.25     ,  416.8828125, 1178.5390625,
       1009.203125 , 2057.0234375, 1027.4765625,  807.984375 ,
        353.3515625, 1126.171875 , 1065.640625 ,  528.796875 ,
       1008.734375 , 1005.3515625,  700.375    ,  571.9609375,
        956.640625 , 1407.9609375, 1434.0234375, 1351.78125  ,
        576.2109375,  549.15625  ,  963.6875   , 1247.2421875,
        309.96875  , 1652.671875 ,  473.8515625, 1084.1

In [36]:
X = df_final
y = train["Price_euros"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1, random_state = 20)
X_test = test_final
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scal = scaler.transform(X_train)
X_test_scal = scaler.transform(X_test)
poly = PolynomialFeatures(degree=2)
poly.fit(X_train_scal)
X_poly_train = poly.transform(X_train_scal)
X_poly_test = poly.transform(X_test_scal)
pol_reg4_reg = Ridge(alpha=0.4)
pol_reg4_reg.fit(X_poly_train, y_train)

predictions = pol_reg4_reg.predict(X_poly_test)
print("Total features shape:", X.shape)
print("Train features shape:", X_train.shape)
print("Train target shape:", y_train.shape)
print("Test features shape:", X_test.shape)
print("Test target shape:", y_test.shape)


Total features shape: (912, 30)
Train features shape: (911, 30)
Train target shape: (911,)
Test features shape: (391, 30)
Test target shape: (1,)


In [37]:
submission = pd.DataFrame({'id': test['id'], 'Price_euros':predictions})
submission.to_csv('../DataFrames/submission.csv', index=False)